In [6]:
%load_ext autoreload
%autoreload 2

%pip install -r requirements.txt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Note: you may need to restart the kernel to use updated packages.


In [7]:
import sys
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision
import torchvision.models as models
from torchvision import transforms
from datasets import load_dataset, concatenate_datasets

In [8]:
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Check for CUDA support
print(f"Is CUDA available? {torch.cuda.is_available()}")

# Set the device
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using device: {device}")


PyTorch version: 2.1.0
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Is CUDA available? False
Using device: mps


In [9]:
# pick which model to load
model_name = "vgg_cifar100" # either "resnet" or "vgg_cifar10" or "vgg_cifar100"

model_path = os.path.join("models", model_name)

In [10]:
from EarlyExitModel import EarlyExitModel
import pickle
import io

# This loads VGG architecture into program memory. Looks out of place but needs to be here. This will get fixed when we rewrite model saving and loading
torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_vgg11_bn", pretrained=True)

# Need a custom unpickler to load the model onto the CPU since it was trained on a GPU and dumped, not saved
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

def load_model(path):
    # Load the model (classifiers trained, but no gates)
    # Load the model onto the CPU
    f = open(os.path.join(model_path, f"full_model_with_exit_gates_alpha_{alpha_no_decimals}.pkl"), "rb")
    model = CPU_Unpickler(f).load()

    # Set the device on the model and its submodules
    model.device = device    
    model = model.to(device)
    model.costs = model.compute_costs_per_exit_module()

    return model



alpha_list = [i / 100 for i in range(0, 101)]
for alpha in alpha_list:
    alpha_no_decimals = str(alpha).replace('.', '_')
    for path in [f"full_model_with_exit_gates_alpha_{alpha_no_decimals}", "final_classifier"]:
        try:
            model = load_model(path)
            print(f"Found model at path {path} to convert")
        except FileNotFoundError:
            continue
        model_name = f"{path}.pth"
        state_dict_path = os.path.join(model_path, model_name)

        torch.save(model.state_dict(), state_dict_path)

Using cache found in /Users/dylanmace/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


Found model at path full_model_with_exit_gates_alpha_0_6 to convert
Found model at path final_classifier to convert
Found model at path full_model_with_exit_gates_alpha_0_8 to convert
Found model at path final_classifier to convert
